In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Load stock data using yfinance
spy = yf.Ticker("spy")
data = pd.DataFrame(spy.history(period="max", interval="1D"))
data = data[["Close", "Volume"]]
m, n = data.shap

In [ ]:
# Calculate MACD and Signal Line
short_window = 12  # 12-period EMA
long_window = 26   # 26-period EMA
signal_window = 9  # 9-period EMA

# Calculate MACD and Signal Line
macd = data['Close'].ewm(span=short_window, adjust=False).mean() - data['Close'].ewm(span=long_window, adjust=False).mean()
signal = macd.ewm(span=signal_window, adjust=False).mean()

# Initialize tracking variables
isBullish_list = []
bullish_counter = 0
bearish_counter = 0

# Loop through the data to check for MACD crossover persistence
for i in range(1, len(macd)):
    if macd.iloc[i] > signal.iloc[i] and macd.iloc[i-1] > signal.iloc[i-1]:  # Golden Cross
        bullish_counter += 1
        bearish_counter = 0
        if bullish_counter >= 4:
            isBullish_list.append(1)  # Bullish
        else:
            isBullish_list.append(2)  # Neutral (before 2 consecutive days)
    elif macd.iloc[i] < signal.iloc[i] and macd.iloc[i-1] < signal.iloc[i-1]:  # Death Cross
        bearish_counter += 1
        bullish_counter = 0
        if bearish_counter >= 4:
            isBullish_list.append(-1)  # Bearish
        else:
            isBullish_list.append(2)  # Neutral (before 2 consecutive days)
    else:
        # Reset counters when there's no persistence of bullish/bearish signal
        bullish_counter = 0
        bearish_counter = 0
        isBullish_list.append(2)  # Neutral state

# Adjust length of isBullish_list for the first element
isBullish_list = [2] + isBullish_list  # Add 2 for the first row since we start from the second row

# Add the isBullish column to the dataset
data['MACD_Status'] = isBullish_list

# Check if the market is in an uptrend for atleast 5 days (TBI)
data['200_MA'] = EMA_200 = data['Close'].rolling(window=200).mean()# Preview the dataset
print(data)